In [12]:
                    #### Importing libraries ####
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import vgg19
import os
import cv2
import matplotlib.pyplot as plt
import imutils
from tqdm.auto import tqdm
import time

In [13]:
def gram_matrix(x):
    x = tf.transpose(x)
    features = tf.reshape(x, (tf.shape(x)[0], -1))
    gram = tf.matmul(features, tf.transpose(features))
    gram = tf.reshape(gram,(tf.shape(gram)[0], -1))
    return gram
def preprocess_image(image_path):
    width, height = keras.preprocessing.image.load_img('Data/Art_Nouveau_Modern/raphael-kirchner_santoy-1900-2.jpg').size
    img_nrows = 256
    img_ncols = int(width * img_nrows / height)
    img = keras.preprocessing.image.load_img(
        image_path, target_size=(img_nrows, img_ncols)
    )
    img = keras.preprocessing.image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return tf.convert_to_tensor(img)
def deprocess_image(x):
    width, height = keras.preprocessing.image.load_img('Data/Art_Nouveau_Modern/raphael-kirchner_santoy-1900-2.jpg').size
    img_nrows = 256
    img_ncols = int(width * img_nrows / height)
    # Util function to convert a tensor into a valid image
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype("uint8")
    return x

In [14]:
model = vgg19.VGG19(weights="imagenet", include_top=False)
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs_dict)

In [15]:
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [16]:
def style_encoding(image_path,feature_extractor):
    image_tensor = preprocess_image(image_path)
    features = feature_extractor(image_tensor)
    layer_features = features['block5_pool']
    gram = gram_matrix(layer_features)
    gram = np.array(tf.reshape(gram, [-1]))
    return gram


def color_encoding(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    features = []
    (h, w) = image.shape[:2]
    (cX, cY) = (int(w * 0.5), int(h * 0.5))
    segments = [(0, cX, 0, cY), (cX, w, 0, cY), (cX, w, cY, h),(0, cX, cY, h)]
    
    (axesX, axesY) = (int(w * 0.75) // 2, int(h * 0.75) // 2)
    
    ellipMask = np.zeros(image.shape[:2], dtype = "uint8")
    
    cv2.ellipse(ellipMask, (cX, cY), (axesX, axesY), 0, 0, 360, 255, -1)
    
    for (startX, endX, startY, endY) in segments:
        
# construct a mask for each corner of the image, subtracting
# the elliptical center from it
        cornerMask = np.zeros(image.shape[:2], dtype = "uint8")
        cv2.rectangle(cornerMask, (startX, startY), (endX, endY), 255, -1)
        cornerMask = cv2.subtract(cornerMask, ellipMask)
        # extract a color histogram from the image, then update the
        # feature vector
        hist = histogram(image, mask = cornerMask)
        features.extend(hist)
    hist = histogram(image, ellipMask)
    features.extend(hist)
    return np.array(features)




def histogram(image, mask):
    # extract a 3D color histogram from the masked region of the
    # image, using the supplied number of bins per channel
    hist = cv2.calcHist([image], [0, 1, 2], mask, (8,10,9),
        [0, 180, 0, 256, 0, 256])
    # normalize the histogram if we are using OpenCV 2.4
    if imutils.is_cv2():
        hist = cv2.normalize(hist).flatten()
    # otherwise handle for OpenCV 3+
    else:
        hist = cv2.normalize(hist, hist).flatten()
    # return the histogram
    return hist

In [17]:
current_dir = os.getcwd()
data_dir = os.path.join('.', 'miniData')
list_of_folders = [i for i in sorted(os.listdir(data_dir)) if not i.startswith('.')]

In [18]:
enc_dict = {'image_array':[],'image_path': [], 'folder_path': [], 'encoding': []}
enc_array = []
enc_dict.keys()

dict_keys(['image_array', 'image_path', 'folder_path', 'encoding'])

In [19]:
ti = []

In [20]:
data_dir

'./miniData'

In [21]:
def encoding(list_of_folders,model):
    for path in tqdm(list_of_folders):
        folder_path = os.path.join(data_dir,path)
        list_images = [image for image in sorted(os.listdir(folder_path)) if not image.startswith('.')]
        for image in tqdm(list_images):
            start_time = time.time()
            image_path = os.path.join(folder_path,image)
            style_encs = style_encoding(image_path,feature_extractor)
            style_end_time = time.time()
            color_encs = color_encoding(image_path)
            color_end_time = time.time() - style_end_time
            encs = np.array(np.concatenate([style_encs,color_encs]),dtype = 'int')
            enc_array.append(encs)
            enc_dict['image_path'].append(os.path.join(folder_path,image))
            enc_dict['folder_path'].append(path)
            ti.append(color_end_time)
            enc_dict['encoding'].append(encs)
    return encs, enc_dict

In [22]:
enc_dict = encoding(list_of_folders,model)

In [23]:
encoding_dict = enc_dict[1]

In [24]:
import pickle
with open('encoding_dict.txt', 'wb') as f:
    pickle.dump(encoding_dict, f)

In [25]:
with open('encoding_dict.txt', 'rb') as handle:
    b = pickle.load(handle)

In [26]:
b['encoding']

[array([44,  0, 25, ...,  0,  0,  0]),
 array([1866,   13,  158, ...,    0,    0,    0]),
 array([4484,   11,  916, ...,    0,    0,    0]),
 array([1148,    0,    0, ...,    0,    0,    0]),
 array([3817,    0,    8, ...,    0,    0,    0]),
 array([2559,   29,  498, ...,    0,    0,    0]),
 array([1633,    0,    4, ...,    0,    0,    0]),
 array([128,   0,   0, ...,   0,   0,   0]),
 array([21941,   158,   266, ...,     0,     0,     0]),
 array([5, 0, 0, ..., 0, 0, 0]),
 array([17140,     0,     0, ...,     0,     0,     0]),
 array([6102,  369,   26, ...,    0,    0,    0]),
 array([10369,  7466,  1004, ...,     0,     0,     0]),
 array([1501,    0,   49, ...,    0,    0,    0]),
 array([557,  21,  17, ...,   0,   0,   0]),
 array([965,   0,   0, ...,   0,   0,   0]),
 array([35047,    13,     0, ...,     0,     0,     0]),
 array([62, 89,  0, ...,  0,  0,  0]),
 array([5991,  103, 1612, ...,    0,    0,    0]),
 array([4288,  499, 7040, ...,    0,    0,    0]),
 array([10524,  